In [11]:
import  warnings  
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report,auc, confusion_matrix,roc_auc_score
import xgboost as xgb
import optuna
from optuna.samplers import TPESampler


In [27]:
#Importation des données
data_test=pd.read_csv("data/test_clean.csv")
data_train=pd.read_csv("data/train_clean.csv")
data_validation=pd.read_csv("data/val_clean.csv")

In [28]:
data_train.head()

,id,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,8105,590.0,27.0,5.0,0.00,2.0,1.0,1.0,27341.63,0.0,0.0,1.0,0.0
1,7940,573.0,24.0,6.0,0.00,2.0,1.0,1.0,106761.47,0.0,0.0,0.0,1.0
2,14446,606.0,38.0,7.0,0.00,2.0,0.0,1.0,117038.96,0.0,0.0,0.0,1.0
3,1280,686.0,49.0,4.0,116465.53,1.0,1.0,0.0,179488.33,1.0,1.0,0.0,0.0
4,3772,634.0,37.0,7.0,0.00,2.0,1.0,1.0,151648.66,0.0,0.0,1.0,1.0


In [29]:
y_train=data_train["Exited"]
X_train=data_train.drop(columns=["Exited"])
y_val=data_validation["Exited"]
X_val=data_validation.drop(columns=["Exited"])


In [30]:
X_train.shape, X_val.shape

((11999, 12), (3000, 12))

In [31]:
def objective(trial):        
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.1),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.4, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 10),  # L1 regularization (alpha)
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 10), # L2 regularization (lambda)
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10), # Min child weight
        'random_state': trial.suggest_int('random_state', 0, 10000),
    }

    model = XGBClassifier(**params)
    model.fit(X_train,y_train)

    y_pred =model. predict_proba(X_val)[:, 1]
    return  roc_auc_score(y_val, y_pred)

study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=42))
study.optimize(objective, n_trials=500)

print("Best parameters:", study.best_params)
print("Best accuracy:", study.best_value)


[I 2024-12-31 00:53:49,294] A new study created in memory with name: no-name-e80108d1-42ac-4b45-95d2-bb453138f068
[I 2024-12-31 00:53:50,010] Trial 0 finished with value: 0.9344860330995074 and parameters: {'n_estimators': 406, 'max_depth': 15, 'learning_rate': 0.07322619478695937, 'subsample': 0.7993292420985183, 'colsample_bytree': 0.5780093202212182, 'colsample_bylevel': 0.49359671220172163, 'gamma': 0.2904180608409973, 'reg_alpha': 8.661761457749352, 'reg_lambda': 6.011150117432088, 'min_child_weight': 8, 'random_state': 205}. Best is trial 0 with value: 0.9344860330995074.
[I 2024-12-31 00:53:50,941] Trial 1 finished with value: 0.9339249065021068 and parameters: {'n_estimators': 972, 'max_depth': 13, 'learning_rate': 0.021312677156759788, 'subsample': 0.5909124836035503, 'colsample_bytree': 0.5917022549267169, 'colsample_bylevel': 0.5825453457757226, 'gamma': 2.6237821581611893, 'reg_alpha': 4.319450186421157, 'reg_lambda': 2.9122914019804194, 'min_child_weight': 7, 'random_state

Best parameters: {'n_estimators': 343, 'max_depth': 8, 'learning_rate': 0.08571583489125943, 'subsample': 0.7817344756772059, 'colsample_bytree': 0.5393607402976373, 'colsample_bylevel': 0.9209678660886534, 'gamma': 2.382541298014462, 'reg_alpha': 4.064590577146683, 'reg_lambda': 8.924597552521014, 'min_child_weight': 5, 'random_state': 8687}
Best accuracy: 0.9352198140345699


In [32]:
study.best_params

{'n_estimators': 343,
 'max_depth': 8,
 'learning_rate': 0.08571583489125943,
 'subsample': 0.7817344756772059,
 'colsample_bytree': 0.5393607402976373,
 'colsample_bylevel': 0.9209678660886534,
 'gamma': 2.382541298014462,
 'reg_alpha': 4.064590577146683,
 'reg_lambda': 8.924597552521014,
 'min_child_weight': 5,
 'random_state': 8687}

In [33]:
params = {'n_estimators': 343,
 'max_depth': 8,
 'learning_rate': 0.08571583489125943,
 'subsample': 0.7817344756772059,
 'colsample_bytree': 0.5393607402976373,
 'colsample_bylevel': 0.9209678660886534,
 'gamma': 2.382541298014462,
 'reg_alpha': 4.064590577146683,
 'reg_lambda': 8.924597552521014,
 'min_child_weight': 5,
 'random_state': 8687}


In [34]:
model_xgb = XGBClassifier(**params)

In [35]:
model_xgb.fit(X_train, y_train)

AttributeError: 'super' object has no attribute '__sklearn_tags__'

AttributeError: 'super' object has no attribute '__sklearn_tags__'

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=0.9209678660886534, colsample_bynode=None,
              colsample_bytree=0.5393607402976373, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=2.382541298014462,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.08571583489125943,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=343, n_jobs=None,
              num_parallel_tree=None, random_state=8687, ...)

In [39]:
predictions_xgb_train_1 =  model_xgb.predict_proba(X_train)[:,1]
# Créez sample_submission avec 'id' et 'Churn_Probability'
sample_submission_test_N = data_train[['id']].copy()  # Copie uniquement la colonne 'id' de data_test
sample_submission_test_N["Exited"] = predictions_xgb_train_1  # Ajout des probabilités de churn
# Optionnel : Exportez sample_submission en CSV
sample_submission_test_N.to_csv("predictions_xgb_train_1.csv", index=False)

In [40]:
predictions_xgb_val_1 =  model_xgb.predict_proba(X_val)[:,1]
# Créez sample_submission avec 'id' et 'Churn_Probability'
sample_submission_test_N = data_validation[['id']].copy()  # Copie uniquement la colonne 'id' de data_test
sample_submission_test_N["Exited"] = predictions_xgb_val_1  # Ajout des probabilités de churn
# Optionnel : Exportez sample_submission en CSV
sample_submission_test_N.to_csv("predictions_xgb_val_1.csv", index=False)

In [43]:
predictions_xgb_test_1 =  model_xgb.predict_proba(data_test)[:,1]
# Créez sample_submission avec 'id' et 'Churn_Probability'
sample_submission_test_N = data_test[['id']].copy()  # Copie uniquement la colonne 'id' de data_test
sample_submission_test_N["Exited"] = predictions_xgb_test_1  # Ajout des probabilités de churn

# Optionnel : Exportez sample_submission en CSV
sample_submission_test_N.to_csv("predictions_xgb_test_1.csv", index=False)

#### Model  Random Forest

In [44]:
from sklearn.ensemble import RandomForestClassifier

In [46]:
def objective(trial):
    # Définir les hyperparamètres à optimiser
    params = {
        'n_estimators': trial.suggest_int("n_estimators", 10, 1500),  # Nombre d'arbres
        'max_depth': trial.suggest_int("max_depth", 2, 32, log=True),  # Profondeur maximale des arbres
        'min_samples_split': trial.suggest_int("min_samples_split", 2, 10),  # Échantillons minimum pour une division
        'min_samples_leaf': trial.suggest_int("min_samples_leaf", 1, 10),  # Échantillons minimum par feuille
        'max_features': trial.suggest_categorical("max_features", ["sqrt", "log2", None]),  # Nombre de caractéristiques
        'max_leaf_nodes': trial.suggest_int("max_leaf_nodes", 10, 1000),  # Nombre maximal de nœuds feuille
        'max_samples': trial.suggest_float("max_samples", 0.1, 1.0),  # Échantillons max pour chaque arbre
        'random_state': trial.suggest_int('random_state', 0, 10000),  # Graine pour reproductibilité
    }
    # Créer le modèle Random Forest avec les hyperparamètres proposés
    model = RandomForestClassifier(
        **params
    )
    
    # Entraîner le modèle sur l'ensemble d'entraînement
    model.fit(X_train, y_train)
    
    # Faire des prédictions sur l'ensemble de validation
    y_val_pred_proba = model.predict_proba(X_val)[:, 1]
    
    # Calculer l'AUC sur les données de validation
    auc = roc_auc_score(y_val, y_val_pred_proba)
    return auc

# Étape 4 : Lancer l'optimisation avec Optuna
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# Étape 5 : Afficher les meilleurs hyperparamètres et résultats
print("Meilleurs hyperparamètres :", study.best_params)
print("Meilleur AUC sur validation :", study.best_value)

[I 2024-12-31 01:28:01,191] A new study created in memory with name: no-name-b7bef832-b050-46cc-873f-056b39a029b5


[I 2024-12-31 01:28:12,052] Trial 0 finished with value: 0.9298222774221038 and parameters: {'n_estimators': 928, 'max_depth': 6, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'log2', 'max_leaf_nodes': 728, 'max_samples': 0.545868085172046, 'random_state': 3542}. Best is trial 0 with value: 0.9298222774221038.
[I 2024-12-31 01:28:13,323] Trial 1 finished with value: 0.8776771865140254 and parameters: {'n_estimators': 173, 'max_depth': 2, 'min_samples_split': 5, 'min_samples_leaf': 6, 'max_features': None, 'max_leaf_nodes': 596, 'max_samples': 0.18532100273845575, 'random_state': 3467}. Best is trial 0 with value: 0.9298222774221038.
[I 2024-12-31 01:28:29,079] Trial 2 finished with value: 0.9315759720856922 and parameters: {'n_estimators': 692, 'max_depth': 25, 'min_samples_split': 10, 'min_samples_leaf': 6, 'max_features': 'log2', 'max_leaf_nodes': 358, 'max_samples': 0.981394480658314, 'random_state': 4890}. Best is trial 2 with value: 0.9315759720856922.
[I 2024-12

Meilleurs hyperparamètres : {'n_estimators': 1160, 'max_depth': 13, 'min_samples_split': 6, 'min_samples_leaf': 10, 'max_features': None, 'max_leaf_nodes': 357, 'max_samples': 0.20848070896998963, 'random_state': 6643}
Meilleur AUC sur validation : 0.9336380775217976


In [47]:
study.best_params

{'n_estimators': 1160,
 'max_depth': 13,
 'min_samples_split': 6,
 'min_samples_leaf': 10,
 'max_features': None,
 'max_leaf_nodes': 357,
 'max_samples': 0.20848070896998963,
 'random_state': 6643}

In [48]:
params_rf = {'n_estimators': 1160,
 'max_depth': 13,
 'min_samples_split': 6,
 'min_samples_leaf': 10,
 'max_features': None,
 'max_leaf_nodes': 357,
 'max_samples': 0.20848070896998963,
 'random_state': 6643}

In [49]:
model_rf  = RandomForestClassifier(**params_rf)

In [51]:
model_rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=13, max_features=None, max_leaf_nodes=357,
                       max_samples=0.20848070896998963, min_samples_leaf=10,
                       min_samples_split=6, n_estimators=1160,
                       random_state=6643)

In [64]:
predictions =  model_rf.predict_proba(data_test.drop(columns=["id"]))[:,1]
# Créez sample_submission avec 'id' et 'Churn_Probability'
sample_submission_test_N = data_test[['id']].copy()  # Copie uniquement la colonne 'id' de data_test
sample_submission_test_N["Exited"] = predictions  # Ajout des probabilités de churn

# Optionnel : Exportez sample_submission en CSV
sample_submission_test_N.to_csv("rf_submission_with_rds_target_encode.csv", index=False)

####  LGBM

In [65]:
import lightgbm as lgb

In [68]:
def objective(trial):
    # Définir les hyperparamètres à optimiser
    param = {
        "objective": "binary",  # Problème de classification binaire
        "metric": "auc",
        "boosting_type": "gbdt",
        "verbosity": -1,
        "n_jobs": -1,
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.3),
        "num_leaves": trial.suggest_int("num_leaves", 16, 64),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 100),
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10.0),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0),
    }

    # Créer un dataset LightGBM
    train_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

    # Entraîner le modèle LightGBM
    model = lgb.train(
        param,
        train_data,
        valid_sets=[val_data],
        num_boost_round=1000,
        # early_stopping_rounds=10,
        # verbose_eval=False,
    )

    # Calculer l'AUC sur l'ensemble de validation
    y_val_pred = model.predict(X_val)
    auc = roc_auc_score(y_val, y_val_pred)
    return auc

# Étape 4 : Lancer l'optimisation avec Optuna
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=500)

# Étape 5 : Afficher les meilleurs hyperparamètres et résultats
print("Meilleurs hyperparamètres :", study.best_params)
print("Meilleur score AUC sur validation :", study.best_value)


[I 2024-12-25 20:21:03,923] A new study created in memory with name: no-name-de38f324-ffbb-41a8-bc35-ea13d04115cf
[I 2024-12-25 20:21:08,875] Trial 0 finished with value: 0.9218300524367932 and parameters: {'learning_rate': 0.21379236895533477, 'num_leaves': 53, 'max_depth': 4, 'min_data_in_leaf': 29, 'lambda_l1': 1.592338490105008, 'lambda_l2': 2.7591404829290067e-08, 'bagging_fraction': 0.7737835634925355, 'feature_fraction': 0.6574488472143314}. Best is trial 0 with value: 0.9218300524367932.
[I 2024-12-25 20:21:15,011] Trial 1 finished with value: 0.923977788854884 and parameters: {'learning_rate': 0.040130238451696366, 'num_leaves': 57, 'max_depth': 7, 'min_data_in_leaf': 79, 'lambda_l1': 0.4444383095677503, 'lambda_l2': 7.439395468485539e-07, 'bagging_fraction': 0.7930347354961425, 'feature_fraction': 0.6994548535068947}. Best is trial 1 with value: 0.923977788854884.
[I 2024-12-25 20:21:20,731] Trial 2 finished with value: 0.9292590622641668 and parameters: {'learning_rate': 0.0

Meilleurs hyperparamètres : {'learning_rate': 0.013448702401249336, 'num_leaves': 58, 'max_depth': 3, 'min_data_in_leaf': 23, 'lambda_l1': 0.04591030003659988, 'lambda_l2': 1.1315696099402986e-07, 'bagging_fraction': 0.650838060664719, 'feature_fraction': 0.7013027931037275}
Meilleur score AUC sur validation : 0.9353841141300867


In [69]:
study.best_params

{'learning_rate': 0.013448702401249336,
 'num_leaves': 58,
 'max_depth': 3,
 'min_data_in_leaf': 23,
 'lambda_l1': 0.04591030003659988,
 'lambda_l2': 1.1315696099402986e-07,
 'bagging_fraction': 0.650838060664719,
 'feature_fraction': 0.7013027931037275}

In [70]:
params_lgbm = {'learning_rate': 0.013448702401249336,
 'num_leaves': 58,
 'max_depth': 3,
 'min_data_in_leaf': 23,
 'lambda_l1': 0.04591030003659988,
 'lambda_l2': 1.1315696099402986e-07,
 'bagging_fraction': 0.650838060664719,
 'feature_fraction': 0.7013027931037275}

In [74]:
model_lgbm = lgb.LGBMClassifier(**params_lgbm)

In [75]:
model_lgbm.fit(X_full, y_full)

LGBMClassifier(bagging_fraction=0.650838060664719,
               feature_fraction=0.7013027931037275,
               lambda_l1=0.04591030003659988, lambda_l2=1.1315696099402986e-07,
               learning_rate=0.013448702401249336, max_depth=3,
               min_data_in_leaf=23, num_leaves=58)

In [76]:
predictions_lgbm =  model_lgbm.predict_proba(data_test.drop(columns=["id"]))[:,1]
# Créez sample_submission avec 'id' et 'Churn_Probability'
sample_submission_lgbm = data_test[['id']].copy()  # Copie uniquement la colonne 'id' de data_test
sample_submission_lgbm["Exited"] = predictions_lgbm  # Ajout des probabilités de churn

# Optionnel : Exportez sample_submission en CSV
sample_submission_lgbm.to_csv("lgbm_submission.csv", index=False)


Stacking

In [7]:
# import predictions
lgbm=pd.read_csv("lgbm_submission.csv").rename(columns={"Exited":"Exited_lgbm"})
rdf=pd.read_csv("rf_submission_with_rds_target_encode.csv").rename(columns={"Exited":"Exited_rdf"})
xgb2 =pd.read_csv("xgb_submission_with_rds_target_encode.csv").rename(columns={"Exited":"Exited_xgb2"})

In [8]:
toto  = lgbm.merge(rdf, on = ["id"]).merge(xgb2, on =["id"])

In [10]:
aucs= [0.93576,  0.9332, 0.9353]

In [11]:
stacking  = lambda row : (row["Exited_lgbm"]*aucs[0] + row["Exited_rdf"]*aucs[1] + row["Exited_xgb2"]*aucs[2]) / sum(aucs)

In [12]:
toto["Exited"] = toto.apply(stacking,axis=1)

In [13]:
toto[["id","Exited"]].to_csv("first_stacking.csv", index=False)

In [6]:
# Import necessary libraries
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
import pandas as pd


# Define models
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'SVC': SVC(probability=True),
    'Decision Tree': DecisionTreeClassifier(),
    'Naive Bayes': GaussianNB(),
    'Linear Discriminant Analysis': LinearDiscriminantAnalysis(),
    'Quadratic Discriminant Analysis': QuadraticDiscriminantAnalysis(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

# Initialize variables for the best model and AUC
best_model = None
best_auc = 0
results = {}

# Train and evaluate all models
for name, model in models.items():
    try:
        model.fit(X_train, y_train)
        predictions = model.predict_proba(X_val)[:, 1] if hasattr(model, "predict_proba") else model.decision_function(X_val)
        auc = roc_auc_score(y_val, predictions)
        results[name] = auc
        if auc > best_auc:
            best_auc = auc
            best_model = model
    except Exception as e:
        # Handle errors for a particular model
        results[name] = f"Error: {str(e)}"

# Sort results by AUC and display
results_sorted = {k: v for k, v in sorted(results.items(), key=lambda item: item[1] if isinstance(item[1], float) else -1, reverse=True)}
results_df = pd.DataFrame(list(results_sorted.items()), columns=['Model', 'AUC'])

# Display results
print("Model Performance:")
print(results_df)

print("\nBest Model:")
print(best_model)

Model Performance:
                              Model       AUC
0                 Gradient Boosting  0.934690
1                          AdaBoost  0.927184
2                     Random Forest  0.924734
3                           XGBoost  0.923060
4   Quadratic Discriminant Analysis  0.900073
5      Linear Discriminant Analysis  0.880508
6               Logistic Regression  0.879513
7                       Naive Bayes  0.832921
8                     Decision Tree  0.776750
9               K-Nearest Neighbors  0.539464
10                              SVC  0.379277

Best Model:
GradientBoostingClassifier()


In [66]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 day 9 hours 32 mins
H2O_cluster_timezone:,Europe/Paris
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,1 month and 27 days
H2O_cluster_name:,H2O_from_python_lamou_isw5lj
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.865 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [67]:
from h2o.frame import H2OFrame
train=H2OFrame(data_train)
val=H2OFrame(data_validation)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [70]:

features = data_train.drop(columns=["Exited"]).columns.tolist()

aml=H2OAutoML(max_models=50, seed=0, 
              exclude_algos=["XGBoost"], 
              nfolds=0, balance_classes=True)

In [69]:
train["Exited"] =train["Exited"].asfactor()
val["Exited"] =val["Exited"].asfactor()

In [71]:
aml.train(x=features,y="Exited",training_frame=train,
           validation_frame=val)
lb = aml.leaderboard
lb.head(rows=lb.nrows)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%


model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
DeepLearning_grid_3_AutoML_3_20241230_01245_model_4,0.932463,0.250641,0.818109,0.149287,0.274303,0.0752419
GBM_grid_1_AutoML_3_20241230_01245_model_10,0.932224,0.250264,0.814582,0.155757,0.274481,0.0753398
DeepLearning_grid_2_AutoML_3_20241230_01245_model_4,0.932097,0.249947,0.814512,0.164153,0.274422,0.0753074
DeepLearning_grid_2_AutoML_3_20241230_01245_model_7,0.931828,0.251294,0.812585,0.155563,0.274779,0.0755033
GBM_grid_1_AutoML_3_20241230_01245_model_13,0.931631,0.253798,0.800898,0.140464,0.277484,0.0769976
DeepLearning_grid_3_AutoML_3_20241230_01245_model_2,0.93138,0.258767,0.815237,0.153686,0.275815,0.0760741
GBM_5_AutoML_3_20241230_01245,0.931257,0.25255,0.810009,0.165,0.275804,0.0760678
DeepLearning_grid_3_AutoML_3_20241230_01245_model_7,0.931179,0.253613,0.812861,0.148853,0.275991,0.0761708
DeepLearning_grid_1_AutoML_3_20241230_01245_model_4,0.931146,0.255144,0.813634,0.157024,0.278316,0.0774596
GBM_1_AutoML_3_20241230_01245,0.93007,0.256784,0.798335,0.160993,0.278138,0.0773605


In [35]:
aml.leader

Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_2_AutoML_2_20241229_231855


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    60                 60                          75045                  7            7            7             58            115           94.95

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.09799355070772285
RMSE: 0.3130392159262524
LogLoss: 0.31028675580717396
Mean Per-Class Error: 0.07728739460809826
AUC: 0.9765879365074275
AUCPR: 0.9753437677896493
Gini: 0.953175873014855

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.1764073581603366
       0     1      Error    Rate
-----  ----  -----  -------  ----------------
0      8647  960    0.0999   (960.0/9607.0)
1      525   9082   0.0546   (525.0/9607.0)
Total  9172  10042  0.0773   (1485.0/19214.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.176407     0.924424  298
max f2                       0.114468     0.949434  324
max f0point5                 0.382365     0.92852   229
max accuracy                 0.176407     0.922713  298
max precision                0.986768     1         0
max recall                   0.0109865    1         388
max specificity              0.986768     1         0
max absolute_mcc             0.176407     0.846293  298
max min_per_class_accuracy   0.217035     0.919017  283
max mean_per_class_accuracy  0.176407     0.922713  298
max tns                      0.986768     9607      0
max fns                      0.986768     9603      0
max fps                      0.00329464   9607      399
max tps                      0.0109865    9607      388
max tnr                      0.986768     1         0
max fnr                      0.986768     0.999584  0
max fpr                      0.00329464   1         399
max tpr                      0.0109865    1         388

Gains/Lift Table: Avg response rate: 50,00 %, avg score: 36,32 %
group    cumulative_data_fraction    lower_threshold    lift       cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  ---------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100448                   0.955339           2          2                  1                0.963732    1                           0.963732            0.0200895       0.0200895                  100       100                0.0200895
2        0.0200895                   0.945876           2          2                  1                0.949936    1                           0.956834            0.0200895       0.040179                   100       100                0.040179
3        0.0300822                   0.941018           2          2                  1                0.943562    1                           0.952425            0.0199854       0.0601645                  100       100                0.0601645
4        0.0400749                   0.934821           2          2                  1                0.937323    1                           0.948659            0.0199854       0.0801499                  100       100                0.0801499
5        0.0501717                   0.928811           2          2                  1                0.931794    1           

In [36]:
preds = aml.predict(val)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [37]:
preds

predict,p0,p1
1,0.0680424,0.931958
1,0.0762428,0.923757
0,0.994463,0.00553718
1,0.141214,0.858786
0,0.803103,0.196897
0,0.950263,0.0497372
0,0.924628,0.0753715
0,0.989009,0.0109908
0,0.697835,0.302165
1,0.48181,0.51819


In [40]:
roc_auc_score(y_val,preds.as_data_frame()["p1"])

c:\Users\lamou\Downloads\M2_MOSEF_S1\Data Mining\challenge\.venv\Lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


np.float64(0.9325534184166484)

In [19]:
# Charger le jeu de test
test = H2OFrame(data_test)

# Vérifier les colonnes attendues
print("Colonnes attendues par le modèle :", features)

# S'assurer que le jeu de test contient uniquement les colonnes nécessaires
test = test[features]  # Supprime les colonnes inutiles s'il y en a


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Colonnes attendues par le modèle : ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary', 'Geography_Germany', 'Geography_Spain', 'Gender_Male']


In [41]:
# Prédire sur le jeu de test
predictions = aml.leader.predict(test)

# Afficher les premières lignes des prédictions
print(predictions.head())


gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
  predict        p0          p1
        0  0.923904  0.0760963
        0  0.991558  0.00844173
        0  0.984647  0.0153532
        1  0.417393  0.582607
        0  0.964614  0.0353856
        0  0.989034  0.0109657
        0  0.996263  0.00373708
        0  0.994196  0.00580371
        0  0.995864  0.00413615
        0  0.996284  0.00371574
[10 rows x 3 columns]



In [42]:
# Extraire uniquement les probabilités de la classe positive
proba_positive = predictions["p1"]

# Afficher les premières probabilités
print(proba_positive.head())


        p1
0.0760963
0.00844173
0.0153532
0.582607
0.0353856
0.0109657
0.00373708
0.00580371
0.00413615
0.00371574
[10 rows x 1 column]



In [43]:
# Convertir les probabilités en DataFrame pandas
proba_positive_df = proba_positive.as_data_frame()
proba_positive_df




c:\Users\lamou\Downloads\M2_MOSEF_S1\Data Mining\challenge\.venv\Lib\site-packages\h2o\frame.py:1983: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


,p1
0,0.076096
1,0.008442
2,0.015353
3,0.582607
4,0.035386
...,...
9995,0.012889
9996,0.848857
9997,0.015847
9998,0.021193


In [45]:
# Créez sample_submission avec 'id' et 'Churn_Probability'
sample_submission_test_N = data_test[['id']].copy()  # Copie uniquement la colonne 'id' de data_test
sample_submission_test_N["Exited"] = proba_positive_df  # Ajout des probabilités de churn

# Optionnel : Exportez sample_submission en CSV
sample_submission_test_N.to_csv("auto2.csv", index=False)

In [46]:
sample_submission_test_N

,id,Exited
0,15000,0.076096
1,15001,0.008442
2,15002,0.015353
3,15003,0.582607
4,15004,0.035386
...,...,...
9995,24995,0.012889
9996,24996,0.848857
9997,24997,0.015847
9998,24998,0.021193
